In [1]:
!pip install smolagents[toolkit] --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.7 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

2025-07-01 17:37:32.801951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751391453.018220      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751391453.082855      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
sentiment_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
from huggingface_hub import login

login()

In [25]:
model_id = "Qwen/Qwen2.5-Coder-14B-Instruct"

In [11]:
ALPHA_VANTAGE_API_KEY = "XF16KZY6ZHWNB09L" 

In [26]:
import re
import requests
import yfinance as yf
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool
from sklearn.linear_model import LogisticRegression
import urllib.parse
@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

import yfinance as yf
import requests
import os
from datetime import datetime, timedelta
from smolagents import tool

@tool
def get_stock_price(ticker: str, historical: bool = False, period: str = "1mo", start_date: str = None, end_date: str = None) -> str:
    """
    Fetches current and/or historical stock price data for the given ticker symbol.

    Args:
        ticker: The stock ticker symbol (e.g., AAPL for Apple, MSFT for Microsoft).
        historical: Whether to include historical data (default: False for current price only).
        period: Time period for historical data. Valid options: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max (default: 1mo).
        start_date: Start date for historical data in YYYY-MM-DD format (optional, overrides period).
        end_date: End date for historical data in YYYY-MM-DD format (optional, defaults to today).

    Returns:
        A string with current price and/or historical data summary, or fallback search result.
    """
    try:
        stock = yf.Ticker(ticker)
        
        if not historical:
            # Get only current price
            price = stock.history(period="1d")['Close'].iloc[-1]
            return f"The current price of {ticker.upper()} is ${price:.2f}."
        
        else:
            # Get historical data
            if start_date and end_date:
                hist_data = stock.history(start=start_date, end=end_date)
            elif start_date:
                hist_data = stock.history(start=start_date)
            else:
                hist_data = stock.history(period=period)
            
            if hist_data.empty:
                return f"No historical data found for {ticker.upper()}."
            
            # Current price
            current_price = hist_data['Close'].iloc[-1]
            
            # Historical summary
            oldest_price = hist_data['Close'].iloc[0]
            highest_price = hist_data['High'].max()
            lowest_price = hist_data['Low'].min()
            avg_volume = hist_data['Volume'].mean()
            
            # Calculate percentage change
            price_change = ((current_price - oldest_price) / oldest_price) * 100
            
            # Date range
            start_str = hist_data.index[0].strftime('%Y-%m-%d')
            end_str = hist_data.index[-1].strftime('%Y-%m-%d')
            
            return (
                f"Historical data for {ticker.upper()} ({start_str} to {end_str}):\n"
                f"Current price: ${current_price:.2f}\n"
                f"Period change: {price_change:+.2f}%\n"
                f"Highest price: ${highest_price:.2f}\n"
                f"Lowest price: ${lowest_price:.2f}\n"
                f"Average daily volume: {avg_volume:,.0f}\n"
                f"Total trading days: {len(hist_data)}"
            )
    
    except Exception:
        # Fallback to Alpha Vantage for current price only
        try:
            
            if historical:
                # Alpha Vantage historical data
                url = (
                    f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY"
                    f"&symbol={ticker}&outputsize=full&apikey={ALPHA_VANTAGE_API_KEY}"
                )
                response = requests.get(url, timeout=10)
                data = response.json()
                
                if "Time Series (Daily)" in data:
                    time_series = data["Time Series (Daily)"]
                    dates = sorted(time_series.keys(), reverse=True)
                    
                    if len(dates) >= 30:  # At least 30 days of data
                        current_price = float(time_series[dates[0]]["4. close"])
                        month_ago_price = float(time_series[dates[29]]["4. close"])
                        price_change = ((current_price - month_ago_price) / month_ago_price) * 100
                        
                        return (
                            f"Historical data for {ticker.upper()} (Alpha Vantage - last 30 days):\n"
                            f"Current price: ${current_price:.2f}\n"
                            f"30-day change: {price_change:+.2f}%"
                        )
                
                return f"Limited historical data available for {ticker.upper()}"
            
            else:
                # Alpha Vantage current price
                url = (
                    f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE"
                    f"&symbol={ticker}&apikey={ALPHA_VANTAGE_API_KEY}"
                )
                response = requests.get(url, timeout=10)
                data = response.json()

                price = float(data["Global Quote"]["05. price"])
                return f"The current price of {ticker.upper()} is ${price:.2f}."
        
        except Exception:
            # Final fallback to web search
            search_query = f"{ticker} current stock price"
            if historical:
                search_query += " historical data"
            search_query += " site:finance.yahoo.com OR site:marketwatch.com"
            
            # Note: You would need to implement visit_webpage or return a search URL
            return f"Unable to fetch data for {ticker.upper()}. Please search manually: {search_query}"



@tool
def market_trend_predictor(ticker: str) -> str:
    """
    Predicts the short-term market trend (up/down) for a stock.
    Falls back to a web search if data unavailable.

    Args:
        ticker: The stock ticker symbol (e.g., AAPL for Apple, MSFT for Microsoft).

    Returns:
        A string indicating the predicted trend or web search fallback result.
    """
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period="6mo", interval="1d")
        if df.empty:
            raise Exception("No yfinance data")

    except Exception:
        try:
            url = (
                f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED"
                f"&symbol={ticker}&outputsize=compact&apikey={ALPHA_VANTAGE_API_KEY}"
            )
            response = requests.get(url)
            data = response.json()

            if "Time Series (Daily)" not in data:
                raise Exception("No Alpha Vantage data")

            df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index").astype(float)
            df.sort_index(inplace=True)
            df.rename(columns={"5. adjusted close": "Close"}, inplace=True)
            df = df[["Close"]]

        except Exception:
            search_query = f"{ticker} short term market trend"
            return visit_webpage(search_query)

    df["pct_change"] = df["Close"].pct_change()
    df["target"] = (df["pct_change"].shift(-1) > 0).astype(int)
    df.dropna(inplace=True)

    if len(df) < 30:
        search_query = f"{ticker} short term market trend"
        return visit_webpage(search_query)

    model = LogisticRegression()
    model.fit(df[["pct_change"]].values, df["target"].values)

    latest_change = df["pct_change"].iloc[-1]
    pred = model.predict([[latest_change]])[0]
    confidence = model.predict_proba([[latest_change]])[0][pred]

    trend = "UP" if pred == 1 else "DOWN"
    return (
        f"Predicted short-term trend for {ticker.upper()}: **{trend}** "
        f"(Confidence: {confidence:.2%})"
    )



@tool
def financial_sentiment_analyzer(news_text: str) -> str:
    """
    Analyzes the sentiment of financial or stock market-related news text.

    Args:
        news_text: The text of the news article, headline, or report.

    Returns:
        The detected sentiment (Positive, Negative, Neutral) along with confidence.
    """
    try:
        result = sentiment_pipeline(news_text)[0]
        label = result['label']
        score = result['score']

        return f"Sentiment: {label} (Confidence: {score:.2%})"
    except Exception as e:
        return f"Error analyzing sentiment: {str(e)}"

@tool
def search_financial_news_web(query: str) -> str:
    """
    Searches for financial news using web search as a fallback.
    
    Args:
        query: The financial topic to search for.
    
    Returns:
        Search results from web search.
    """
    try:
        import requests
        from markdownify import markdownify
        import re
        
        search_url = f"https://www.google.com/search?q={urllib.parse.quote(query + ' latest financial news')}"
        response = requests.get(search_url, timeout=10)
        response.raise_for_status()
        
        markdown_content = markdownify(response.text).strip()
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
        
        return markdown_content
        
    except Exception as e:
        return f"Error searching web for '{query}': {str(e)}"



In [8]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

Hugging Face - Wikipedia

[Jump to content](#bodyContent)

Main menu

Main menu

move to sidebar
hide

Navigation

* [Main page](/wiki/Main_Page "Visit the main page [z]")
* [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")
* [Current events](/wiki/Portal:Current_events "Articles related to current events")
* [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")
* [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia and how it works")
* [Conta


In [27]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
)

model = InferenceClientModel(model_id=model_id)

web_agent = ToolCallingAgent(
    tools=[WebSearchTool(), visit_webpage],
    model=model,
    max_steps=2,
    name="web_search_agent",
    description="Runs web searches for you.",
)

finance_agent = ToolCallingAgent(
    tools=[get_stock_price, market_trend_predictor, financial_sentiment_analyzer, search_financial_news_web],
    model=model,
    
    max_steps=2,
    name="finance_agent",
    description="Handles stock prices and market summary tasks.",
)

In [28]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent, finance_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)

In [29]:
answer = manager_agent.run(
    "give me a rundown of the latest news in the stock market today"
)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ give me a rundown of the latest news in the stock market today                                                  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-14B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task_description = "Provide a comprehensive rundown of the latest news in the stock market today, including key  
  trends, major events, and notable performances of various stocks and indices."                                   
  additional_args = {}                                                                                             
                                                                                                                   
  latest_stock_news = finance_agent(task=task_description, additional_args=additional_args)                        
  print(latest_stock_news)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────── New run - finance_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'finance_agent'.                                                                   │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Provide a comprehensive rundown of the latest news in the stock market today, including key trends, major       │
│ events, and notable performances of various stocks and indices.                                                 │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {}.                                                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-14B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 2: Duration 39.06 seconds| Input tokens: 4,746 | Output tokens: 236]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Summary of the latest stock market news                                                                        
  summary = """                                                                                                    
  Key trends in the stock market today include:                                                                    
  - Major indices such as the Dow Jones, NASDAQ, and S&P 500 showed mixed performance.                             
  - Notable stock performances were observed in companies like Tesla.                                              
  - Significant market events included Federal Reserve Chairman Jerome Powell's comments and tensions between      
  Donald Trump and Elon Musk.                                                                                      
  """                                                                                                              
                                                                                                                   
  final_answer(summary)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Key trends in the stock market today include:
- Major indices such as the Dow Jones, NASDAQ, and S&P 500 showed mixed performance.
- Notable stock performances were observed in companies like Tesla.
- Significant market events included Federal Reserve Chairman Jerome Powell's comments and tensions between Donald 
Trump and Elon Musk.

[Step 3: Duration 11.41 seconds| Input tokens: 7,798 | Output tokens: 368]